In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Input,Model
from keras.layers import Conv2D, BatchNormalization, Activation, Add, Conv2DTranspose, ZeroPadding2D, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow_addons.layers import InstanceNormalization
from imageio import imread
from cv2 import resize
import numpy as np
import pandas as pd
import cv2
import matplotlib.image as mpimg
from PIL import Image
from tensorflow.keras.callbacks import TensorBoard
%pylab inline

In [ ]:
train = pd.read_csv('../input/nyu-depth-v2/nyu_data/data/nyu2_train.csv',header=None)
test = pd.read_csv('../input/nyu-depth-v2/nyu_data/data/nyu2_test.csv',header=None)

In [ ]:
train[0][0]

In [ ]:
path = '../input/nyu-depth-v2/nyu_data/'

In [ ]:
img = mpimg.imread(path+train[0][1])
plt.imshow(img)

In [ ]:
img = Image.open(path+train[1][1])

img.width,img.height

In [ ]:
def residual_block(x):
    res = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    res = BatchNormalization(axis=3, momentum=0.9, epsilon=1e-5)(res)
    res = Activation('relu')(res)
    
    res = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(res)
    res = BatchNormalization(axis=3, momentum=0.9, epsilon=1e-5)(res)
    
    return Add()([res, x])

In [ ]:
def build_generator():
    input_shape = (128,128,3)
    residual_blocks = 6
    input_layer = Input(shape=input_shape)
    
    x = Conv2D(filters=32, kernel_size=7, strides=1, padding='same')(input_layer)
    x = InstanceNormalization(axis=1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=64,kernel_size=3,strides=2,padding='same')(x)
    x = InstanceNormalization(axis=1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(x)
    x = InstanceNormalization(axis=1)(x)
    x = Activation('relu')(x)
    
    for _ in range(residual_blocks):
        x = residual_block(x)
        
    x = Conv2DTranspose(filters=64, kernel_size=3, strides=2,padding='same',use_bias=False)(x)
    x = InstanceNormalization(axis=1)(x)
    x = Activation('relu')(x)
    
    x = Conv2DTranspose(filters=32, kernel_size=3, strides=2,padding='same',use_bias=False)(x)
    x = InstanceNormalization(axis=1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=3,kernel_size=7,strides=1,padding='same')(x)
    output = Activation('tanh')(x)
    
    model = Model(inputs=[input_layer], outputs=[output])
    return model

In [ ]:
def build_discriminator():
    input_shape = (128,128,3)
    hidden_layers = 3
    
    input_layer = Input(shape=input_shape)
    
    x = ZeroPadding2D(padding=(1,1))(input_layer)
    
    x = Conv2D(filters=64,kernel_size=4,strides=2,padding='valid')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = ZeroPadding2D(padding=(1,1))(x)
    
    for i in range(1,hidden_layers+1):
        x = Conv2D(filters=2**i*64, kernel_size=4,strides=2,padding='valid')(x)
        x = InstanceNormalization(axis=1)(x)
        x = LeakyReLU(alpha=0.2)(x)
        
        x = ZeroPadding2D(padding=(1,1))(x)
        
    output = Conv2D(filters=1,kernel_size=4,strides=1,activation='sigmoid')(x)
    
    model = Model(inputs=[input_layer], outputs=[output])
    
    return model

In [ ]:
batch_size = 8
epochs = 100

In [ ]:
def load_images(path,inp_paths,dep_paths):
    images_inp = list(path+inp_paths)
    images_dep = list(path+dep_paths)

    all_iminp = []
    all_imdep = []

    for index,filename in enumerate(images_inp):
        inp = imread(filename)
        dep = imread(images_dep[index])

        inp = resize(inp, (128,128,3))
        dep = resize(dep, (128,128,3))

        if np.random.random()>0.5:
            inp = np.fliplr(inp)
            dep = np.fliplr(dep)

        all_iminp.append(inp)
        all_imdep.append(dep)

    all_iminp = np.array(all_iminp)/127.5-1
    all_imdep = np.array(all_imdep)/127.5-1
    
    return all_iminp,all_imdep

In [ ]:
optimizer = Adam(0.0002,0.5)

In [ ]:
discriminator_inp = build_discriminator()
discriminator_dep = build_discriminator()

In [ ]:
discriminator_inp.compile(loss = 'mse',
                          optimizer = optimizer,
                          metrics = ['accuracy'])

discriminator_dep.compile(loss = 'mse',
                          optimizer = optimizer,
                          metrics = ['accuracy'])

In [ ]:
generator_inptodep = build_generator()
generator_deptoinp = build_generator()

In [ ]:
input_inp = Input((128,128,3))
input_dep = Input((128,128,3))

In [ ]:
generated_inp = generator_deptoinp(input_dep)
generated_dep = generator_inptodep(input_inp)

In [ ]:
regenerated_inp = generator_deptoinp(generated_dep)
regenerated_dep = generator_inptodep(generated_inp)

In [ ]:
generated_inpid = generator_deptoinp(input_inp)
generated_depid = generator_inptodep(input_dep)

In [ ]:
discriminator_inp.trainable = False
discriminator_dep.trainable = False

In [ ]:
prob_inp = discriminator_inp(generated_inp)
prob_dep = discriminator_dep(generated_dep)

In [ ]:
adversarial_model = Model(inputs=[input_inp,input_dep],outputs=[prob_inp,prob_dep,regenerated_inp,regenerated_dep,generated_inpid,generated_depid])

In [ ]:
adversarial_model.compile(loss = ['mse','mse','mae','mae','mae','mae'],
                          loss_weights=[1,1,10.0,10.0,1.0,1.0],
                          optimizer=optimizer)

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(time.time()),write_images=True,write_grads=True,write_graph=True)
tensorboard.set_model(generator_inptodep)
tensorboard.set_model(generator_deptoinp)
tensorboard.set_model(discriminator_inp)
tensorboard.set_model(discriminator_dep)

In [ ]:
real_labels = np.ones((batch_size,7,7,1))
fake_labels = np.zeros((batch_size,7,7,1))

In [ ]:
discriminator_inp.trainable = True
discriminator_dep.trainable = True

In [ ]:
num_batches = int(len(train[0])/batch_size)
for epoch in range(epochs):
    print('Epoch:{}'.format(epoch))
    
    dis_losses = []
    gen_losses = []

    for index in range(30):
        batch_inp = train[0][index*batch_size : (index+1)*batch_size]
        batch_dep = train[1][index*batch_size : (index+1)*batch_size]

        batch_inp,batch_dep = load_images(path,batch_inp,batch_dep)
        generated_dep = generator_inptodep(batch_inp)
        generated_inp = generator_deptoinp(batch_dep)

        daloss_inp1 = discriminator_inp.train_on_batch(batch_inp,real_labels)
        daloss_inp2 = discriminator_dep.train_on_batch(generated_inp,fake_labels)

        dbloss_dep1 = discriminator_dep.train_on_batch(batch_dep,real_labels)
        dbloss_dep2 = discriminator_dep.train_on_batch(generated_dep,fake_labels)
        
        d_loss = 0.5*np.add(0.5*np.add(daloss_inp1,daloss_inp2),0.5*np.add(dbloss_dep1,dbloss_dep2))
        
        g_loss = adversarial_model.train_on_batch([batch_inp,batch_dep],[real_labels,real_labels,batch_inp,batch_dep,batch_inp,batch_dep])
    
    dis_losses.append(d_loss)
    gen_losses.append(g_loss)
    
  
        
        
print(batch_inp.shape)

In [ ]:
generated_dep[1]

In [ ]:
generated_dep=generated_dep.numpy()
fig,ax = plt.subplots(8,1,figsize=(50,50))

for i in range(8):
    ax[i].imshow(generated_dep[i].squeeze(),cmap='Greys')

In [ ]:
 batch_inp,batch_out=load(path,batch_inp,batch_dep)
        generated_dep = generator_inptodep(batch_inp)
        generated_inp = generator_deptoinp(batch_dep)

        daloss_inp1 = discriminator_inp.train_on_batch(batch_inp,real_labels)
        daloss_inp2 = discriminator_dep.train_on_batch(generated_inp,fake_labels)

        dbloss_dep1 = discriminator_dep.train_on_batch(batch_dep,real_labels)
        dbloss_dep2 = discriminator_dep.train_on_batch(generated_dep,fake_labels)
        
        d_loss = 0.5*np.add(0.5*np.add(daloss_inp1,daloss_inp2),0.5*np.add(dbloss_dep1,dbloss_dep2))